In [1]:
#files to import
import numpy as np
import cv2
from collections import defaultdict
import time

Kalman equation 1: 
Contain the state space model of position and velocity in x and y coordinates that is row and column


In [9]:
#kalman filter
def k_predict(x,P,FPS=30):
    delta_t = 1
    #x is state 
    #p is uncertainity covariance
    F = np.array([[1,0,delta_t,0],[0,1,0,delta_t],[0,0,1,0],[0,0,0,1]])#state transistion matrix
    x_pred = np.dot(F,x)
    p_pred = np.dot(np.dot(F,P),F.T)
    return x_pred,p_pred

def k_update(z,x_pred,p_pred):
    R = np.identity(4)#measurement uncertainity assume to be 1
    I = np.identity(4) #identity matrix
    H = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]])#measurment function contain all the information velocity as well as position in x and y coord
    y = z - np.dot(H,x_pred)#residul or error
    s = np.dot(np.dot(H,p_pred),H.T)+R
    k = np.dot(np.dot(p_pred,H.T),np.linalg.inv(s))#kalman gain
    x_next = x_pred+np.dot(k,y) #corrected state
    p_next = np.dot((I-np.dot(k,H)),p_pred) #corrected covariance
    return x_next,p_next

In [10]:
#feature parameter for the shi tomasi algo
feature_params = dict(maxCorners=50,
                    qualityLevel=0.2,
                    minDistance = 10,
                    blockSize = 10)

#parameters for lucasKanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

#create the color
#color  = np.random.randint(0,255,(300,3))

check_id = 0 #it is the id given to each object
main_id = 0
object_ids = dict() #it is the dictionary which can contains the id and the coordinate
disappeared = dict() #it is the dictionary which contains the number of time the object is dissappeared
max_disappeared = 10 #till 10 frames the object will be followed up if it is not visible after 10 frames it will be remove
#state and noise initialization for n corner points
state = defaultdict(np.array)
P_cov = defaultdict(np.array)

max_points = 30
points = defaultdict(list)
points_kalman = defaultdict(list)

##TODO
#Edit the filename want to track
cap = cv2.VideoCapture("filename")
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
#create the feature points
fp = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

#state initialization
for i,fppp in enumerate(fp):
    (c,d) = fppp.ravel()
    check_id+=1
    object_ids[check_id] = object_ids.get(check_id,(c,d))
    disappeared[check_id] = disappeared.get(check_id,0)
    points[check_id].append((c,d))
    points_kalman[check_id].append((c,d))
    state[check_id] = np.array([c,d,0,0]).reshape(4,1) #[x,y,vx,vy]
    P_cov[check_id] = np.array([[1,0,0,0],[0,1,0,0],[0,0,10,0],[0,0,0,10]])
#print(fp)
while(cap.isOpened()):
    ret,frame = cap.read()
    if ret==True:
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        tp,st,err = cv2.calcOpticalFlowPyrLK(old_gray,gray_frame,fp,None,**lk_params)
        good_new = tp[st==1]
        good_old = fp[st==1]
        image = frame.copy()
        image1 = image.copy()
        image2 = image.copy()

        for i,(old,new) in enumerate(zip(good_old,good_new)):
            a,b = old.ravel().astype(int)
            c,d = new.ravel().astype(int)
            z = np.array([c,d,c-a,d-b]).reshape(4,1)
            try:
                main_id = [key for key,values in object_ids.items() if values[0]==a and values[1]==b][0]
            except Exception as e:
                main_id=-1
                pass

            if main_id<0:
                check_id+=1
                object_ids[check_id] = object_ids.get(check_id,(c,d))
                disappeared[check_id] = disappeared.get(check_id,0)
                points[check_id].append((c,d))
                main_id=check_id
            else:
                object_ids[main_id]=(c,d)
                points[main_id].append((c,d))
                if len(points[main_id])>max_points:
                    points[main_id].pop(0)
            for newp,point in enumerate(points[main_id]):
                if newp>=len(points[main_id])-2:
                    break
                (rpx,rpy) = point
                (rnx,rny) = points[main_id][newp+1]
                image = cv2.line(frame,(rpx,rpy),(rnx,rny),(255,0,0),2)
            image1 = cv2.circle(image1,(c,d),2,(255,0,0),2)
            image1 = cv2.putText(image1,"x:"+str(c),(c-2,d-2),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,0,0),1)
            #for iterate in range(10):
            x_predict,p_predict = k_predict(state[main_id],P_cov[main_id])
            x_next,p_next = k_update(z,x_predict,p_predict)
            pfpx = int(x_predict[0][0])
            pfpy = int(x_predict[1][0])
            pfpvx = int(x_predict[2][0])
            pfpvy = int(x_predict[3][0])
            #pfpx = int(x_next[0][0])
            #pfpy = int(x_next[1][0])
            
            image2 = cv2.circle(image2,(pfpx,pfpy),2,(0,0,255),2)
            image2 = cv2.putText(image2,"x:"+str(pfpx),(pfpx-2,pfpy-2),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,255),1)
            state[main_id] = x_next
            P_cov[main_id] = p_next
        #new_fp = cv2.goodFeaturesToTrack(gray_frame, mask = None, **feature_params)
        #for nfp in new_fp:
        #    (nfpx,nfpy) = nfp.ravel()
        #    image = cv2.circle(image,(nfpx,nfpy),2,(255,0,0),2)
        fp = good_new.reshape(-1,1,2)
        old_gray = gray_frame.copy()
        cv2.imshow('frame',frame)
        #cv2.imshow('main',image)
        cv2.imshow('original',image1)
        cv2.imshow('kalman',image2)
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()